In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import os

# URLs to scrape data from
urls = [
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U01.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S01.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S02.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S03.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S04.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U02.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S26.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U03.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S06.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S07.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S08.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U08.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S27.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S10.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S11.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U09.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U06.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S12.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S13.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S14.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S15.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S16.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S17.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S18.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U07.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S19.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S20.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S21.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S22.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S29.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S23.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S24.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S28.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S25.htm"
]

# Mapping of state codes to state names
state_code_to_name = {
    "U01": "Andaman and Nicobar Islands",
    "S01": "Andhra Pradesh",
    "S02": "Arunachal Pradesh",
    "S03": "Assam",
    "S04": "Bihar",
    "U02": "Chandigarh",
    "S26": "Chhattisgarh",
    "U03": "Dadra and Nagar Haveli",
    "S05": "Goa",
    "S06": "Gujarat",
    "S07": "Haryana",
    "S08": "Himachal Pradesh",
    "U08": "Jammu and Kashmir",
    "S27": "Jharkhand",
    "S10": "Karnataka",
    "S11": "Kerala",
    "U09": "Lakshadweep",
    "U06": "Delhi",
    "S12": "Madhya Pradesh",
    "S13": "Maharashtra",
    "S14": "Manipur",
    "S15": "Meghalaya",
    "S16": "Mizoram",
    "S17": "Nagaland",
    "U05": "Puducherry",
    "S18": "Punjab",
    "U07": "Daman and Diu",
    "S19": "Rajasthan",
    "S20": "Sikkim",
    "S21": "Tamil Nadu",
    "S22": "Telangana",
    "S29": "Tripura",
    "S23": "Uttar Pradesh",
    "S24": "Uttarakhand",
    "S28": "West Bengal",
    "S25": "Odisha"
}

# Define regions
regions = {
    "North": ["Chandigarh", "Delhi", "Haryana", "Himachal Pradesh", "Jammu and Kashmir", "Punjab", "Rajasthan", "Uttarakhand", "Uttar Pradesh"],
    "South": ["Andhra Pradesh", "Karnataka", "Kerala", "Puducherry", "Tamil Nadu", "Telangana"],
    "East": ["Arunachal Pradesh", "Assam", "Bihar", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Sikkim", "Tripura", "West Bengal"],
    "West": ["Goa", "Gujarat", "Maharashtra", "Dadra and Nagar Haveli", "Daman and Diu"],
    "Central": ["Chhattisgarh", "Madhya Pradesh", "Jharkhand"]
}

parent_directory = "region-wise-party-distribution"
if not os.path.exists(parent_directory):
    os.makedirs(parent_directory)

def extract_state_name(url):
    state_code = url.split("-")[-1].split(".")[0]
    state_name = state_code_to_name.get(state_code, state_code)  # Use mapping to get state name
    return state_name, state_code

def scrape_seat_distribution(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    party_seats = []
    grid_boxes = soup.find_all("div", class_="grid-box")
    for box in grid_boxes:
        party_name = box.find("h4").text
        seats_won = box.find("h2").text
        party_seats.append({"Party": party_name, "Seats": int(seats_won)})
    
    return pd.DataFrame(party_seats)

def save_charts(df, region_name):
    directory = os.path.join(parent_directory, region_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Bar chart
    bar_chart_path = os.path.join(directory, f"{region_name}_party_distribution_bar.png")
    df.plot(kind="bar", x="Party", y="Seats", title=f"Party-wise Seat Distribution ({region_name})")
    plt.ylabel("Seats Won")
    plt.savefig(bar_chart_path)
    plt.close()
    
    # Pie chart
    pie_chart_path = os.path.join(directory, f"{region_name}_party_distribution_pie.png")
    df.plot(kind="pie", y="Seats", labels=df["Party"], autopct="%1.1f%%", title=f"Party-wise Seat Distribution ({region_name})")
    plt.ylabel("")
    plt.savefig(pie_chart_path)
    plt.close()

region_wise_data = {region: {} for region in regions}

for url in urls:
    state_name, state_code = extract_state_name(url)
    df_seat_distribution = scrape_seat_distribution(url)
    for region, states in regions.items():
        if state_name in states:
            for _, row in df_seat_distribution.iterrows():
                party = row['Party']
                seats = row['Seats']
                if party not in region_wise_data[region]:
                    region_wise_data[region][party] = 0
                region_wise_data[region][party] += seats

for region, data in region_wise_data.items():
    df_region = pd.DataFrame(list(data.items()), columns=["Party", "Seats"])
    save_charts(df_region, region)
    print(f"Charts saved for {region}")

print("Scraping and chart generation completed.")


Charts saved for North
Charts saved for South
Charts saved for East
Charts saved for West
Charts saved for Central
Scraping and chart generation completed.
